<div align="center">
<font color="blue"><h1>GDAL - Introduction to Raster Data</h1>
<h3> Aug 14-18, 2020 @ UNAVCO</h3></font></div>

In this tutorial, we will demonstrate the use of GDAL software for working geospatial data in general and with radar data products generated using the ISCE software. We will start with very basics of geospatial data representations and work through various common operations involving geospatial imagery.

At the end of this general introduction, we will use GDAL to manipulate Standard Unwrapped Interferogram products from the ARIA/GRFN archive.

## 0 Notebook setup

In [6]:
# Always run this cell at the start of each session
import os
from osgeo import gdal, osr

if 'nb_dir' not in globals():
    nb_dir = os.getcwd()

## 1 Download datasets for notebook

We will download the datasets used in this notebook. 

### Dataset 1: SRTM DEM tile in geotiff format over California from opentopography

In [2]:
!wget -nc https://cloud.sdsc.edu/v1/AUTH_opentopography/Raster/SRTM_GL1_Ellip/SRTM_GL1_Ellip_srtm/North/North_30_60/N33W119_wgs84.tif

--2020-07-26 19:54:49--  https://cloud.sdsc.edu/v1/AUTH_opentopography/Raster/SRTM_GL1_Ellip/SRTM_GL1_Ellip_srtm/North/North_30_60/N33W119_wgs84.tif
Resolving cloud.sdsc.edu (cloud.sdsc.edu)... 132.249.120.48, 132.249.120.78
Connecting to cloud.sdsc.edu (cloud.sdsc.edu)|132.249.120.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6410371 (6.1M) [image/tiff]
Saving to: ‘N33W119_wgs84.tif’

N33W119_wgs84.tif   100%[===================>]   6.11M  6.95MB/s    in 0.9s    

2020-07-26 19:54:50 (6.95 MB/s) - ‘N33W119_wgs84.tif’ saved [6410371/6410371]



### Dataset 2: Greenland DEM in netcdf format from Univ of Montana

In [3]:
!wget -nc http://websrv.cs.umt.edu/isis/images/a/ab/Greenland1km.nc

--2020-07-26 19:54:59--  http://websrv.cs.umt.edu/isis/images/a/ab/Greenland1km.nc
Resolving websrv.cs.umt.edu (websrv.cs.umt.edu)... 150.131.192.24
Connecting to websrv.cs.umt.edu (websrv.cs.umt.edu)|150.131.192.24|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84110176 (80M) [application/x-netcdf]
Saving to: ‘Greenland1km.nc’

Greenland1km.nc     100%[===================>]  80.21M  6.85MB/s    in 18s     

2020-07-26 19:55:17 (4.37 MB/s) - ‘Greenland1km.nc’ saved [84110176/84110176]



### Dataset 3: Southern California DEM in GMT format from UC Santa Barbara

In [4]:
!wget -nc https://projects.eri.ucsb.edu/mapcat/mapftp/18.grd

--2020-07-26 19:55:20--  https://projects.eri.ucsb.edu/mapcat/mapftp/18.grd
Resolving projects.eri.ucsb.edu (projects.eri.ucsb.edu)... 128.111.100.88
Connecting to projects.eri.ucsb.edu (projects.eri.ucsb.edu)|128.111.100.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2888672 (2.8M)
Saving to: ‘18.grd’

18.grd              100%[===================>]   2.75M  6.45MB/s    in 0.4s    

2020-07-26 19:55:21 (6.45 MB/s) - ‘18.grd’ saved [2888672/2888672]



### Dataset 4: USGS DEM over Bushkill, PA from PSU

In [5]:
!wget -nc https://courseware.e-education.psu.edu/downloads/natureofgeoinfo/DEM.zip

--2020-07-26 19:55:25--  https://courseware.e-education.psu.edu/downloads/natureofgeoinfo/DEM.zip
Resolving courseware.e-education.psu.edu (courseware.e-education.psu.edu)... 146.186.26.53
Connecting to courseware.e-education.psu.edu (courseware.e-education.psu.edu)|146.186.26.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2600801 (2.5M) [application/zip]
Saving to: ‘DEM.zip’

DEM.zip             100%[===================>]   2.48M  2.57MB/s    in 1.0s    

2020-07-26 19:55:27 (2.57 MB/s) - ‘DEM.zip’ saved [2600801/2600801]



### Dataset 5: An example ISCE stripmapApp.py run folder

We can use any stripmapApp.py output folder. In our case, we will reuse the one from our earlier notebook using ALOS PALSAR data.

In [ ]:
#If you are on UNAVCO Jupyterlab setup, link to outputs from stripmap notebook
#If not feel free to use any stripmapApp.py run from ISCE
!ln -s /home/jovyan/work/Stripmap/Hawaii_ALOS1 stripmap

## 2. GIS - Geographic Information System

GIS is a framework for organizing, managing, visualizing and analyzing geospatial data. As geophysicists, geologists or geodesists, we all use these set of tools to work with data on maps. Any data that can be plotted on a map can be classified as **geospatial** data. 

Most popular free GIS tools include:

1. Generic Mapping Tools (GMT) http://gmt.soest.hawaii.edu/
2. Geospatial Data Abstraction Library (GDAL) http://www.gdal.org/
3. QGIS https://qgis.org/
4. PROJ.4 / pyproj https://proj4.org/
5. Others like Shapely, Fiona, Caropy, Basemap etc 

In this tutorial, we will focus on the <u>Geospatial Data Abstraction Library (GDAL)</u>.

### 1.1 Types of geospatial data

Geospatial data can generally be classified into 2 types

#### Raster data

From the ARCGIS webpage:

In its simplest form, a raster consists of a matrix of cells (or pixels) organized into rows and columns (or a grid) where each cell contains a value representing information. Typical examples of raster imagery include:

1. Weather radar maps
2. Satellite imagery

Clearly, from the exercises that we have been doing with SAR data - these types of imagery fall under raster datasets. This tutorial will focus on raster data manipulation.

#### Vector data

Vector data refers to datasets that attempt to represent information using goemetric constructs like points, line and polygons. Typical examples of vector data include:

1. Highways, country boundaries etc
2. Earthquake locations from a catalog
3. Polygon shaped features like coastlines etc

With SAR/InSAR, these types of datasets tend to be used in an ancillary fashion to assist in interpretation. This type of data will not discussed in detail as such in this tutorial.

### 1.2 Why GDAL?

* Free and Open Source (https://github.com/OSGeo/gdal)
* Support for over 80+ Image formats and map projections
* Command line as well as C/C++/Python/R API
* Used extensively by worlds large geospatial data services
* Extensive test suite and active developer community
* GDAL also includes extensive support for vector datasets (Not a topic for this tutorial)

<b><font color="blue">Note:<br>
    This notebook is specifically to be used with GDAL 3.x
    </font></b>

## 2. Raster Data Model in GDAL

GDAL has a very clearly defined Data Model. For a detailed description of all the aspects of the Data Model - see http://www.gdal.org/gdal_datamodel.html. In this tutorial, we will focus on some important aspects of this model.

### a. Raster Bands

A GDAL-comptabile raster image can consist of many 2D matrices contained in it as long as

1. They are of the same size 
2. They represent information from the same points, i.e the map coordinates are the same.
3. Some data formats inherently allow the different bands to be of different datatypes, but this is not very common.

Let's look at examples of single and multi-band images using the **gdalinfo** utility.

In [7]:
#Lets look at one downloaded SRTM tile
# This is a single band file

!gdalinfo N33W119_wgs84.tif

Driver: GTiff/GeoTIFF
Files: N33W119_wgs84.tif
Size is 3601, 3601
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (-119.000138888888884,34.000138888888891)
Pixel Size = (0.000277777777778,-0.000277777777778)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  COMPRESSION=DEFLATE
  INTERLEAVE=BAND
Corner Coordinates:
Upper Left  (-119.0001389,  34.0001389) (119d 0' 0.50"W, 34d 0' 0.50"N)
Lower Left  (-119.0001389,  32.9998611) (119d 0' 0.50"W, 32d59'59.50"N)
Upper R

In [ ]:
#Lets look at a coherence product generated by ISCE
# This is a two band file

!gdalinfo stripmap/interferogram/topophase.cor.geo.vrt

In [ ]:
###How do we get this information programatically
ds = gdal.Open('stripmap/interferogram/topophase.cor.geo.vrt', gdal.GA_ReadOnly)
print('Bands = ', ds.RasterCount)
print('Dimensions = ', ds.RasterXSize, ' x ', ds.RasterYSize)
ds = None

### b. Data types

The following data types are supported by GDAL:

1. Byte 
2. UInt16 
3. Int16 
4. UInt32 
5. Int32 
6. Float32
7. Float64
8. **CInt16**
9. **CInt32**
10. **CFloat32**
11. **CFloat64** 

Fact that GDAL includes support for complex numbers makes it significantly more relevant than other GIS systems for use with radar datasets.

In [ ]:
!gdalinfo stripmap/interferogram/topophase.flat.vrt

In [ ]:
##Programmatic method of querying data type
ds = gdal.Open('18.grd', gdal.GA_ReadOnly)
band = ds.GetRasterBand(1)
print(gdal.GetDataTypeName(band.DataType))
band = None
ds = None

### c. Coordinate System

This is where GDAL really stands out. There are numerous conventions used globally for representing the coordinate system for map data. Here are some popular ones:

1. PROJ4 - https://proj4.org/usage/projections.html
2. USGS - Collection of 18 numbers, extremely unusable
3. OGC WKT (defacto standard) - http://docs.opengeospatial.org/is/12-063r5/12-063r5.html
4. EPSG codes (Easy to use) - http://spatialreference.org/ref/epsg/
5. XML etc

We will use the **gdalsrsinfo** to demonstrate the complications involved in supporting these coordinate sytems.

In [8]:
##Simple WGS84 lat/lon projection has EPSG code 4326.
##To look at all equivalent representations of this system
!gdalsrsinfo -o all EPSG:4326


PROJ.4 : +proj=longlat +datum=WGS84 +no_defs

OGC WKT1 :
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AXIS["Latitude",NORTH],
    AXIS["Longitude",EAST],
    AUTHORITY["EPSG","4326"]]

OGC WKT2:2015 :
GEODCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    AREA["World"],
    BBOX[-90,-180,90,180],
    ID["EPSG

In [9]:
##Lets look at UTM zone 20N
!gdalsrsinfo -o all EPSG:32620


PROJ.4 : +proj=utm +zone=20 +datum=WGS84 +units=m +no_defs

OGC WKT1 :
PROJCS["WGS 84 / UTM zone 20N",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",-63],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","32620"]]

OGC WKT2:2015 :
PROJCRS["WGS 84 / UTM zone 20N",
    BASEGEODCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,2

You can also use **gdalsrsinfo** to query the coordinate system of a given raster product in a specific format.

In [10]:
##Get projection information for a raster file
!gdalsrsinfo -o proj4  N33W119_wgs84.tif


+proj=longlat +datum=WGS84 +no_defs



In [12]:
###Programmatic method of querying spatial reference
ds = gdal.Open('N33W119_wgs84.tif', gdal.GA_ReadOnly)
srs = ds.GetProjectionRef()
ds = None
print(srs)

ref = osr.SpatialReference()
ref.ImportFromWkt(srs)
ref.ExportToProj4()

GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]


'+proj=longlat +datum=WGS84 +no_defs'

When you use **gdalsrsinfo** on a raster that does not have the coordinate system information, or as is often the case with radar products in radar geometry - it should raise an error.

In [13]:
##This should raise an error
!gdalsrsinfo 18.grd

ERROR 1: ERROR - failed to load SRS definition from 18.grd


In [ ]:
##This should raise an error
!gdalsrsinfo stripmap/topophase.cor

### d. Coordinates on a map

In the subsection above, we discussed coordinate systems. In this section, we talk about how GDAL associates a line/pixel location in a raster with map coordinates.

GDAL uses a simple affine system to translate (pixel, line) location to (X,Y) coordinates in map coordinates. The affine system is represented by 6 double precision numbers collectively known as the ''GeoTransform''.

**Top-left** map coordinate of a pixel represented by pixel number (P) and line number (L) in C/Python indexing is given by

$$\begin{equation*}
\left[ \begin{array}{c}
X_{left} \\
Y_{top}
\end{array} \right] = \left[ \begin{array}{c}
G_0 \\
G_3 \end{array}\right] + \left[ \begin{array}{cc}
G_1 & G_2 \\
G_4 & G_5 
\end{array} \right] \cdot \left[ \begin{array}{c}
P \\
L 
\end{array} \right]
\end{equation*}
$$


Here is the exact manner in which pixel indices are interpreted by GDAL
![Raster Layout](notebook_images/Raster_layout.png)

In [14]:
##To confirm the Top-left interpretation, lets look gdalinfo output for the SRTM tile again
!gdalinfo N33W119_wgs84.tif

Driver: GTiff/GeoTIFF
Files: N33W119_wgs84.tif
Size is 3601, 3601
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (-119.000138888888884,34.000138888888891)
Pixel Size = (0.000277777777778,-0.000277777777778)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  COMPRESSION=DEFLATE
  INTERLEAVE=BAND
Corner Coordinates:
Upper Left  (-119.0001389,  34.0001389) (119d 0' 0.50"W, 34d 0' 0.50"N)
Lower Left  (-119.0001389,  32.9998611) (119d 0' 0.50"W, 32d59'59.50"N)
Upper R

In [15]:
##An image can include coordinate information even if the coordinate system information is missing
##But this is highly discouraged

!gdalinfo 18.grd

Driver: GMT/GMT NetCDF Grid Format
Files: 18.grd
Size is 1201, 601
Origin = (-121.002499999999998,35.502499999999998)
Pixel Size = (0.005000000000000,-0.005000000000000)
Corner Coordinates:
Upper Left  (-121.0025000,  35.5025000) 
Lower Left  (-121.0025000,  32.4975000) 
Upper Right (-114.9975000,  35.5025000) 
Lower Right (-114.9975000,  32.4975000) 
Center      (-118.0000000,  34.0000000) 
Band 1 Block=1201x1 Type=Float32, ColorInterp=Undefined


In [16]:
###Programmatic method of querying the geotransform

ds = gdal.Open('N33W119_wgs84.tif', gdal.GA_ReadOnly)
print(ds.GetGeoTransform())
ds = None

(-119.00013888888888, 0.0002777777777777778, 0.0, 34.00013888888889, 0.0, -0.0002777777777777778)


Most geospatial data formats prefer North-up orientation for storing map data. Hence, it is very common to see a negative $G_5$ term in the GeoTransform representation.

In [ ]:
###For a dataset with no coordinate system / map information - i.e, plain raster
ds = gdal.Open('stripmap/interferogram/topophase.cor.vrt', gdal.GA_ReadOnly)
print(ds.GetGeoTransform())
ds = None

### e. Ground Control Points (GCPs)

GCPs are another mechanism to associate geocoordinate information with raster images. GCPs represent an explicit mapping from a (pixel, line) location to (X,Y) location in map coordinates. These are usually not the most accurate representations, but for some applications are more than sufficient. For example, radar images acquired over very flat terrain or oceans can be geocoded reasonably well using GCPs. We are not going to use this feature in our tutorials, but one should be aware that this information is available with most SLC products delivered by different agencies.

Sentinel-1 SLC and GRD geotiff files are all annotated with GCPs. Shown below is an example output of gdalinfo on a Sentinel-1 SLC image

```bash
> gdalinfo s1a-iw1-slc-vv-20160408t091355-20160408t091428-010728-01001f-001.tiff
Driver: GTiff/GeoTIFF
Files: s1a-iw1-slc-vv-20160408t091355-20160408t091428-010728-01001f-001.tiff
Size is 20665, 17916
Coordinate System is `'
GCP Projection =
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["unnamed",6378137,298.2572235604902,
            AUTHORITY["EPSG","4326"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0],
    UNIT["degree",0.0174532925199433],
    AUTHORITY["EPSG","4326"]]
GCP[  0]: Id=1, Info=
          (0,0) -> (130.457799786776,32.2907336141953,478.460773384615)
GCP[  1]: Id=2, Info=
          (1034,0) -> (130.506581179458,32.2986697089719,478.460773384615)
GCP[  2]: Id=3, Info=
          (2068,0) -> (130.554913356166,32.3065128120191,478.460773384615)
GCP[  3]: Id=4, Info=
          (3102,0) -> (130.602812666404,32.3142661230211,478.460773384615)
...
GCP[270]: Id=271, Info=
          (18612,17915) -> (130.826037469855,34.4318305450677,0)
GCP[271]: Id=272, Info=
          (19646,17915) -> (130.869763720912,34.438454503529,0)
GCP[272]: Id=273, Info=
          (20664,17915) -> (130.912557438728,34.4449210788505,0)
Metadata:
  AREA_OR_POINT=Area
  TIFFTAG_DATETIME=2016:04:08 16:04:16
  TIFFTAG_IMAGEDESCRIPTION=Sentinel-1A IW SLC L1
  TIFFTAG_SOFTWARE=Sentinel-1 IPF 002.62
Image Structure Metadata:
  INTERLEAVE=BAND
Corner Coordinates:
Upper Left  (    0.0,    0.0)
Lower Left  (    0.0,17916.0)
Upper Right (20665.0,    0.0)
Lower Right (20665.0,17916.0)
Center      (10332.5, 8958.0)
Band 1 Block=20665x1 Type=CInt16, ColorInterp=Gray
```

### f. Subdatasets

There are some file formats like netcdf or HDF5 that allow multiple datasets to be packaged together. In such cases, GDAL will report these different layers as sub-datasets. We will explore this will the downloaded Greenland dataset.

In [17]:
!gdalinfo Greenland1km.nc

Warning 1: No UNIDATA NC_GLOBAL:Conventions attribute
Driver: netCDF/Network Common Data Format
Files: Greenland1km.nc
Size is 512, 512
Metadata:
  NC_GLOBAL#comments=The data in this file was interpolated onto a 1km grid using natural neighbor interpolation which is based on Delauney triangulation.  This was accomplished using the function natgridd (see http://www.ncarg.ucar.edu//ngmath/natgrid/nnhome.html) from the National Center for Atmospheric Research (NCAR).  Natgrid is part of the NCAR Command Language (NCL) package which can be downloaded via http://www.ncl.ucar.edu/Download/
  NC_GLOBAL#creator=Glen D. Granzow
  NC_GLOBAL#Date=1 November 2010
  NC_GLOBAL#institution=University of Montana
  NC_GLOBAL#title=Greenland 1 km Data Set
Subdatasets:
  SUBDATASET_1_NAME=NETCDF:"Greenland1km.nc":presprcp
  SUBDATASET_1_DESC=[1x2801x1501] lwe_precipitation_rate (32-bit floating-point)
  SUBDATASET_2_NAME=NETCDF:"Greenland1km.nc":bheatflx
  SUBDATASET_2_DESC=[1x2801x1501] upward_geotherm

To access a specific sub-dataset within the netcdf file, you can use the "SUBDATASET\_\*\_NAME" atrtibute reported by gdalinfo.

In [18]:
!gdalinfo NETCDF:"Greenland1km.nc":topg

Warning 1: No UNIDATA NC_GLOBAL:Conventions attribute
Driver: netCDF/Network Common Data Format
Files: Greenland1km.nc
Size is 1501, 2801
Coordinate System is:
PROJCRS["unnamed",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["unnamed",
        METHOD["Polar Stereographic (variant B)",
            ID["EPSG",9829]],
        PARAMETER["Latitude of standard parallel",71,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8832]],
        PARAMETER["Longitude of origin",-39,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8833]],
        PARAMETER["False easting",0,
            LENGTHUNIT["unknown",1],
            ID["EPSG",8806]],
        PARAMETER["False northing",0,
            LENGTHUNIT["unknown",1],

In [19]:
!gdalsrsinfo NETCDF:"Greenland1km.nc":topg


PROJ.4 : +proj=stere +lat_0=90 +lat_ts=71 +lon_0=-39 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs

OGC WKT2:2018 :
PROJCRS["unnamed",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["unnamed",
        METHOD["Polar Stereographic (variant B)",
            ID["EPSG",9829]],
        PARAMETER["Latitude of standard parallel",71,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8832]],
        PARAMETER["Longitude of origin",-39,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8833]],
        PARAMETER["False easting",0,
            LENGTHUNIT["unknown",1],
            ID["EPSG",8806]],
        PARAMETER["False northing",0,
            LENGTHUNIT["unknown",1],
            ID["EPSG",8807]]],
    CS[Carte

### g. Other features

There are other useful features that GDAL provides for working with raster data. One can read more about these here: 
http://www.gdal.org/gdal_datamodel.html

## 3. Raster data formats

We have already mentioned that GDAL can support over 80+ data formats. GDAL tries to provide a unified interface to reading data from these different formats. Support for different formats depends on what libraries were used when building the GDAL library. You can find out the various raster formats that GDAL supports using **gdalinfo**.

In [20]:
#List all supported formats in your GDAL build
!gdalinfo --formats

Supported Formats:
  VRT -raster,multidimensional raster- (rw+v): Virtual Raster
  DERIVED -raster- (ro): Derived datasets using VRT pixel functions
  GTiff -raster- (rw+vs): GeoTIFF
  COG -raster- (wv): Cloud optimized GeoTIFF generator
  NITF -raster- (rw+vs): National Imagery Transmission Format
  RPFTOC -raster- (rovs): Raster Product Format TOC format
  ECRGTOC -raster- (rovs): ECRG TOC format
  HFA -raster- (rw+v): Erdas Imagine Images (.img)
  SAR_CEOS -raster- (rov): CEOS SAR Image
  CEOS -raster- (rov): CEOS Image
  JAXAPALSAR -raster- (rov): JAXA PALSAR Product Reader (Level 1.1/1.5)
  GFF -raster- (rov): Ground-based SAR Applications Testbed File Format (.gff)
  ELAS -raster- (rw+v): ELAS
  AIG -raster- (rov): Arc/Info Binary Grid
  AAIGrid -raster- (rwv): Arc/Info ASCII Grid
  GRASSASCIIGrid -raster- (rov): GRASS ASCII Grid
  ISG -raster- (rov): International Service for the Geoid
  SDTS -raster- (rov): SDTS Raster
  DTED -raster- (rwv): DTED Elevation Raster
  PNG -raster-

The capability for each format is indicated with a string in the output and here is the corresponding legend:

1. r: read
2. ro: read only
3. w: write "Create dataset by copying another"
4. w+: write with support to update "create writable dataset"
5. s: supports subdatasets
6. v: supports virtual IO - eg. /vsimem/

Virtual features are special and we will revisit this later in this tutorial.

To demonstrate that different formats can be used to represent the same data, we have included a folder called **DEM** that contains the SRTM 90m DEM for the same tile in 3 different formats - SRTM native hgt format (from LPDAAC), GMT format (from GMTSAR's dem service) and Geotiff format (OpenTopography). You can verify that the contents are same by running the command.

In [21]:
!gdalinfo -stats N33W119_wgs84.tif

Driver: GTiff/GeoTIFF
Files: N33W119_wgs84.tif
Size is 3601, 3601
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (-119.000138888888884,34.000138888888891)
Pixel Size = (0.000277777777778,-0.000277777777778)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  COMPRESSION=DEFLATE
  INTERLEAVE=BAND
Corner Coordinates:
Upper Left  (-119.0001389,  34.0001389) (119d 0' 0.50"W, 34d 0' 0.50"N)
Lower Left  (-119.0001389,  32.9998611) (119d 0' 0.50"W, 32d59'59.50"N)
Upper R

In [22]:
!gdalinfo -stats 18.grd

Driver: GMT/GMT NetCDF Grid Format
Files: 18.grd
Size is 1201, 601
Origin = (-121.002499999999998,35.502499999999998)
Pixel Size = (0.005000000000000,-0.005000000000000)
Corner Coordinates:
Upper Left  (-121.0025000,  35.5025000) 
Lower Left  (-121.0025000,  32.4975000) 
Upper Right (-114.9975000,  35.5025000) 
Lower Right (-114.9975000,  32.4975000) 
Center      (-118.0000000,  34.0000000) 
Band 1 Block=1201x1 Type=Float32, ColorInterp=Undefined
  Minimum=-74.586, Maximum=3411.414, Mean=443.527, StdDev=504.287
  Metadata:
    STATISTICS_MAXIMUM=3411.4138183594
    STATISTICS_MEAN=443.52650734192
    STATISTICS_MINIMUM=-74.586204528809
    STATISTICS_STDDEV=504.28678716346
    STATISTICS_VALID_PERCENT=100


<br>
<div class="alert alert-info">
<b>Note :</b>
As you can see that GDAL supports a wide range of data formats and provides a unified interface. If you plan to work with raster datasets, we strongly recommend that you use one of these supported formats as a basis. There needs to be a really compelling reason to pick a different format.
</div>



## 4. Virtual File Systems

GDAL does not necessarily your need data to be stored on a local file. It provides a file like interface to various data sources and these are referred to as Virtual Formats. Such systems are called Virtual File Systems. For a complete list of Virtual File Systems supported by GDAL - see http://www.gdal.org/gdal_virtual_file_systems.html

We will look at a couple of examples of how these work. 

### a. vsicurl

This is meant to be used with datasets that are served online via http / https/ ftp . To enable this feature, GDAL needs to be built with "curl" support. Here is an example of how we would use this feature

In [23]:
##Let's look at opentopography's ellipsoid corrected dataset
!gdalinfo /vsicurl/https://cloud.sdsc.edu/v1/AUTH_opentopography/Raster/SRTM_GL1_Ellip/SRTM_GL1_Ellip_srtm/North/North_30_60/N34W120_wgs84.tif

Driver: GTiff/GeoTIFF
Files: /vsicurl/https://cloud.sdsc.edu/v1/AUTH_opentopography/Raster/SRTM_GL1_Ellip/SRTM_GL1_Ellip_srtm/North/North_30_60/N34W120_wgs84.tif
Size is 3601, 3601
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (-120.000138888888884,35.000138888888891)
Pixel Size = (0.000277777777778,-0.000277777777778)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  COMPRESSION=DEFLATE
  INTERLEAVE=BAND
Corner Coordinates:
Upper Left  (-120.0001389,  35.00013

In [24]:
#This is an example of working with Amazon's Landsat archive
!gdalinfo /vsicurl/http://landsat-pds.s3.amazonaws.com/L8/001/003/LC80010032014272LGN00/LC80010032014272LGN00_B1.TIF    

Driver: GTiff/GeoTIFF
Files: /vsicurl/http://landsat-pds.s3.amazonaws.com/L8/001/003/LC80010032014272LGN00/LC80010032014272LGN00_B1.TIF
       /vsicurl/http://landsat-pds.s3.amazonaws.com/L8/001/003/LC80010032014272LGN00/LC80010032014272LGN00_B1.TIF.ovr
       /vsicurl/http://landsat-pds.s3.amazonaws.com/L8/001/003/LC80010032014272LGN00/LC80010032014272LGN00_MTL.txt
Size is 9131, 9121
Coordinate System is:
PROJCRS["WGS 84 / UTM zone 29N",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["UTM zone 29N",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",-9,


### b. vsizip / vsitar / vsigzip

In [25]:
#Look at the DEM downloaded from the PSU site
!zipinfo DEM.zip

Archive:  DEM.zip
Zip file size: 2600801 bytes, number of entries: 12
drwx---     2.0 fat        0 b- stor 04-Nov-17 10:01 DEM_10m/
-rw-a--     2.0 fat  9687040 t- defN 03-Mar-18 06:39 DEM_10m/bushkill_pa.dem
-rw-a--     2.0 fat    29376 t- defN 03-Apr-05 18:41 DEM_10m/bushkill_pa.html
-rw-a--     2.0 fat    23087 t- defN 03-Apr-05 18:41 DEM_10m/bushkill_pa.text
-rw-a--     2.0 fat      973 t- defN 03-Feb-07 15:48 DEM_10m/bushkill_pa.txt
-rw-a--     2.0 fat    20758 t- defN 03-Apr-05 18:41 DEM_10m/bushkill_pa.xml
drwx---     2.0 fat        0 b- stor 04-Nov-17 10:01 DEM_30m/
-rw-a--     2.0 fat    26279 t- defN 03-Aug-28 08:11 DEM_30m/bushkill_dem_30m_metadata.html
drwx---     2.0 fat        0 b- stor 04-Nov-17 10:01 DEM_30m/bushkill_dem_30m_metadata_files/
-rw-a--     2.0 fat      104 b- defN 03-Aug-28 08:10 DEM_30m/bushkill_dem_30m_metadata_files/disk.gif
-rw-ah-     2.0 fat     3072 b- defN 03-Aug-28 08:33 DEM_30m/bushkill_dem_30m_metadata_files/Thumbs.db
-rw-a--     2.0 fat  1077248

In [26]:
###Directly interact with data inside zip / tar files
!gdalinfo /vsizip/DEM.zip/DEM_30m/bushkill_pa.dem

Driver: USGSDEM/USGS Optional ASCII DEM (and CDED)
Files: /vsizip/DEM.zip/DEM_30m/bushkill_pa.dem
Size is 350, 465
Coordinate System is:
PROJCRS["unnamed",
    BASEGEOGCRS["NAD27",
        DATUM["North American Datum 1927",
            ELLIPSOID["Clarke 1866",6378206.4,294.978698213898,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4267]],
    CONVERSION["UTM zone 18N",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",-75,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["False easting",500000,
            LENGTHUNIT["metre",

### c. vsis3/ vsigs/ vsiaz

Lot of the new updates in GDAL are focused on extending support to data stored on cloud platforms - allowing users to interact with small portions of data without having to download everything.

#### Option 1: Using vsicurl and cookies with .netrc file

In this case, we will reuse the .netrc file settings that we had setup earlier for automated DEM downloads from LP DAAC. If you haven't done this, complete Step 2 from the link shown here: 
https://wiki.earthdata.nasa.gov/display/EL/How+To+Access+Data+With+cURL+And+Wget

In [28]:
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','asf_cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', 'asf_cookies.txt')
results = gdal.Info("/vsicurl/https://grfn.asf.alaska.edu/door/download/S1-GUNW-D-R-160-tops-20190710_20190628-162436-20935N_18926N-PP-6a53-v2_0_2.nc")
print(results)

Driver: HDF5/Hierarchical Data Format Release 5
Files: /vsicurl/https://grfn.asf.alaska.edu/door/download/S1-GUNW-D-R-160-tops-20190710_20190628-162436-20935N_18926N-PP-6a53-v2_0_2.nc
Size is 512, 512
Metadata:
  author=David Bekaert
  Conventions=CF-1.6
  crs_polygon_long_name=crs_polygon
  crs_polygon_spatial_ref=GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]
  crs_polygon_standard_name=crs_polygon
  institution=Jet Propulsion Laboratory
  matchup_CLASS=DIMENSION_SCALE
  matchup_NAME=This is a netCDF dimension but not a netCDF variable.         0
  matchup_REFERENCE_LIST=
  matchup__Netcdf4Dimid=0 
  ogr_geometry_field=productBoundingBox
  ogr_layer_name=productBoundingBox
  ogr_layer_type=POLYGON
  product_type=UNW GEO IFG
  references=https://aria.jpl.nasa.gov/
  science_grid

More about how to do interact with data that ASF is hosting on the Amazon Cloud in later tutorials ... 

### d. vsimem

These are feature that allows one to use in memory arrays just like other GDAL datasets. Here is an example:

In [29]:
##Get memory driver
memdrv = gdal.GetDriverByName('MEM')

##Open source dataset in read only mode
ds = gdal.Open('N33W119_wgs84.tif', gdal.GA_ReadOnly)

##Create an in-memory copy
memds = memdrv.CreateCopy('dummyname', ds)

##Close source
ds = None

##Use copy for operations
print(memds.GetProjection())
print(memds.GetGeoTransform())

##Close memory copy
memds = None

GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
(-119.00013888888888, 0.0002777777777777778, 0.0, 34.00013888888889, 0.0, -0.0002777777777777778)


## 5. Virtual Raster Table (VRT) format

Virtual Raster Table (VRT) format extends the idea of virtualization to actual disks on file. VRT is a custom format supported by GDAL that is similar to "links" or "shortcuts" on operating systems - except that it has more functionality than just pointing to other data sources.

Often, when working with raster datasets, we tend to generate a fairly large volume of intermediate products due to operations like cropping, multilooking and masking. VRT's offer the option of building workflow pipelines that generate such products on the fly. We will rely extensively on this feature for our time-series exercises later. 

Details on GDAL's VRT feature can be found here: http://www.gdal.org/gdal_vrttut.html

You may have already noticed that ISCE generates a **.vrt** file for every raster product that is generated by the workflows. We will be looking at VRTs in greater detail as part of the other notebooks on raster data manipulation. But here are some short examples that demonstrate the capabilities of this format.

### a. Cropping of datasets

ISCE uses GDAL VRT files to virtually extract a single burst from the Sentinel-1 SAFE products. From our TOPS processing tutorial, you will remember that Sentinel-1 TOPS mode data is delivered as SAFE products which contains TIFF files for each imaged swath. Each swath consists of number of bursts or scans. 

To avoid replicating this data on disk again, we use a simple VRT file to point to the location of burst data within the TIFF files. For example, if you were to look at the contents of master/IW1/burst04.slc.vrt, you will see something like 

```xml
<VRTDataset rasterXSize="20665" rasterYSize="1493">
    <VRTRasterBand dataType="CFloat32" band="1">
        <NoDataValue>0.0</NoDataValue>
        <SimpleSource>
            <SourceFilename relativeToVRT="1">/vsizip//Users/agram/data/isce/japan/data/S1A_IW_SLC__1SSV_20160408T091355_20160408T091430_010728_01001F_83EB.zip/S1A_IW_SLC__1SSV_20160408T091355_20160408T091430_010728_01001F_83EB.SAFE/measurement/s1a-iw1-slc-vv-20160408t091355-20160408t091428-010728-01001f-001.tiff</SourceFilename>
            <SourceBand>1</SourceBand>
            <SourceProperties RasterXSize="20665" RasterYSize="17916" DataType="CInt16"/>
            <SrcRect xOff="130" yOff="4498" xSize="20468" ySize="1458"/>
            <DstRect xOff="130" yOff="19" xSize="20468" ySize="1458"/>
        </SimpleSource>
    </VRTRasterBand>
</VRTDataset>
```

For clarity, lets annotate the vrt file to describe what is happening. Note that VRT format is built on top of XML. So, we will use XML's comment feature to annotate the XML for your understanding.

```xml
<!--Size of the virtual dataset-->
<VRTDataset rasterXSize="20665" rasterYSize="1493">
    <!-- Band 1 of the dataset and its data type -->
    <VRTRasterBand dataType="CFloat32" band="1">
        <!--No data value for this dataset-->
        <NoDataValue>0.0</NoDataValue>
        <!--We use a simple source to suggest that we don't alter contents of the data in source-->
        <SimpleSource>
            <!--Name of the source - notice that we point to a TIFF inside a zip file -->
            <SourceFilename relativeToVRT="1">/vsizip//Users/agram/data/isce/japan/data/S1A_IW_SLC__1SSV_20160408T091355_20160408T091430_010728_01001F_83EB.zip/S1A_IW_SLC__1SSV_20160408T091355_20160408T091430_010728_01001F_83EB.SAFE/measurement/s1a-iw1-slc-vv-20160408t091355-20160408t091428-010728-01001f-001.tiff</SourceFilename>
            <!--Source may contain multiple bands. Indicate band from source -->
            <SourceBand>1</SourceBand>
            <!--Properties of the source. This is optional. GDAL will figure it out on the fly if not provided, but will be slow as it actually has to read the source to figure this out. Note that source is of type CInt16 -->
            <SourceProperties RasterXSize="20665" RasterYSize="17916" DataType="CInt16"/>
            <!--Indicate spatial subset of source data in pixels and lines -->
            <SrcRect xOff="130" yOff="4498" xSize="20468" ySize="1458"/>
            <!--Indicate spatial location of destination-->
            <DstRect xOff="130" yOff="19" xSize="20468" ySize="1458"/>
        </SimpleSource>
    </VRTRasterBand>
</VRTDataset>
```

### b. Modify some metadata

Remember the GMT format dataset from the exercise above? Remember that it was tagged with spatial coordinate information but not with coordinate system metadata. We can fix that quickly by writing a VRT that points to the file with the coordinate information included in it.

In [30]:
!gdal_translate -of VRT -a_srs EPSG:4326 18.grd UCSB_18arcsec.dem.vrt

Input file size is 1201, 601


In [31]:
!cat UCSB_18arcsec.dem.vrt

<VRTDataset rasterXSize="1201" rasterYSize="601">
  <SRS dataAxisToSRSAxisMapping="2,1">GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]</SRS>
  <GeoTransform> -1.2100250000000000e+02,  5.0000000000000001e-03,  0.0000000000000000e+00,  3.5502499999999998e+01,  0.0000000000000000e+00, -5.0000000000000001e-03</GeoTransform>
  <VRTRasterBand dataType="Float32" band="1">
    <Metadata>
      <MDI key="STATISTICS_MAXIMUM">3411.4138183594</MDI>
      <MDI key="STATISTICS_MEAN">443.52650734192</MDI>
      <MDI key="STATISTICS_MINIMUM">-74.586204528809</MDI>
      <MDI key="STATISTICS_STDDEV">504.28678716346</MDI>
      <MDI key="STATISTICS_VALID_PERCENT">100</MDI>
    </Metadata>
    <SimpleSource>
      <SourceFilename relativeToVRT="1">18.gr

In [32]:
!gdalinfo UCSB_18arcsec.dem.vrt

Driver: VRT/Virtual Raster
Files: UCSB_18arcsec.dem.vrt
       18.grd
Size is 1201, 601
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (-121.002499999999998,35.502499999999998)
Pixel Size = (0.005000000000000,-0.005000000000000)
Corner Coordinates:
Upper Left  (-121.0025000,  35.5025000) (121d 0' 9.00"W, 35d30' 9.00"N)
Lower Left  (-121.0025000,  32.4975000) (121d 0' 9.00"W, 32d29'51.00"N)
Upper Right (-114.9975000,  35.5025000) (114d59'51.00"W, 35d30' 9.00"N)
Lower Righ

### c. Mosaicking

A very common use case is the need to put together many tiles of rasters to create a wide-area map. One such example is the SRTM DEM. Remember that this data is distributed as 1 deg x 1 deg tiles. With the VRT format, one can put together a single file that represents the global collection of the SRTM DEM with the appropriate parts of the raster pointing to the corresponding tiles. For example, here is how the Open Topography system uses VRTs. Allow sometime for the following command to execute before moving to next cell...

In [33]:
!gdalinfo -nofl /vsicurl/https://cloud.sdsc.edu/v1/AUTH_opentopography/Raster/SRTM_GL1_Ellip/SRTM_GL1_Ellip_srtm.vrt

Driver: VRT/Virtual Raster
Files: /vsicurl/https://cloud.sdsc.edu/v1/AUTH_opentopography/Raster/SRTM_GL1_Ellip/SRTM_GL1_Ellip_srtm.vrt
Size is 1296001, 417601
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (-180.000138888888898,60.000138888888891)
Pixel Size = (0.000277777777778,-0.000277777777778)
Corner Coordinates:
Upper Left  (-180.0001389,  60.0001389) (180d 0' 0.50"W, 60d 0' 0.50"N)
Lower Left  (-180.0001389, -56.0001389) (180d 0' 0.50"W, 56d 0' 0.50"S)
Upper Right

### d. Derived datasets

Often, we end up in situations where we need to manipulate the data slightly for use with other analysis tools. For example, derive power in dB or amplitude or phase from coregistered SLCs. Generating these derived products could use up a lot of disk space. VRTs provide a mechanism called **pixel functions** to do these transformations on the fly. Set of pixel functions available are (current list):

* "real": extract real part from a single raster band (just a copy if the input is non-complex)
* "imag": extract imaginary part from a single raster band (0 for non-complex)
* "complex": make a complex band merging two bands used as real and imag values
* "mod": extract module from a single raster band (real or complex)
* "phase": extract phase from a single raster band [-PI,PI] (0 or PI for non-complex)
* "conj": computes the complex conjugate of a single raster band (just a copy if the input is non-complex)
* "sum": sum 2 or more raster bands
* "diff": computes the difference between 2 raster bands (b1 - b2)
* "mul": multiply 2 or more raster bands
* "cmul": multiply the first band for the complex conjugate of the second
* "inv": inverse (1./x). Note: no check is performed on zero division
* "intensity": computes the intensity Re(x*conj(x)) of a single raster band (real or complex)
* "sqrt": perform the square root of a single raster band (real only)
* "log10": compute the logarithm (base 10) of the abs of a single raster band (real or complex): log10( abs( x ) )
* "dB": perform conversion to dB of the abs of a single raster band (real or complex): 20. * log10( abs( x ) )
* "dB2amp": perform scale conversion from logarithmic to linear (amplitude) (i.e. 10 ^ ( x / 20 ) ) of a single raster band (real only)
* "dB2pow": perform scale conversion from logarithmic to linear (power) (i.e. 10 ^ ( x / 10 ) ) of a single raster band (real only)


Note that this list is very relevant for SAR/InSAR users ;)

# <u> You are now ready to start exploring the Geocoded, Unwrapped data products from ARIA using GDAL </u>